In [1]:
!pip install tensorflow-macos tensorflow-metal


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install scikit-learn matplotlib numpy


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.datasets import fashion_mnist

print("TensorFlow:", tf.__version__)

TensorFlow: 2.16.2


In [4]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = fashion_mnist.load_data()

print("Full train shape:", x_train_full.shape)
print("Full test shape:", x_test_full.shape)

x_train_small = x_train_full[:50]
y_train_small = y_train_full[:50]

x_test_small = x_test_full[:50]
y_test_small = y_test_full[:50]

print("Small train shape:", x_train_small.shape)
print("Small test shape:", x_test_small.shape)


Full train shape: (60000, 28, 28)
Full test shape: (10000, 28, 28)
Small train shape: (50, 28, 28)
Small test shape: (50, 28, 28)


In [5]:
x_train_small = x_train_small.astype("float32") / 255.0
x_test_small  = x_test_small.astype("float32") / 255.0

x_train_small = np.expand_dims(x_train_small, axis=-1)
x_test_small  = np.expand_dims(x_test_small, axis=-1)

print("Train small reshaped:", x_train_small.shape)
print("Test small reshaped:", x_test_small.shape)

x_train, x_val, y_train, y_val = train_test_split(
    x_train_small,
    y_train_small,
    test_size=0.3,
    random_state=42,
    stratify=y_train_small
)

print("x_train shape:", x_train.shape)
print("x_val shape:", x_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)


Train small reshaped: (50, 28, 28, 1)
Test small reshaped: (50, 28, 28, 1)
x_train shape: (35, 28, 28, 1)
x_val shape: (15, 28, 28, 1)
y_train shape: (35,)
y_val shape: (15,)


In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("TensorFlow:", tf.__version__)


TensorFlow: 2.16.2


In [8]:
def create_cnn_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(10, activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

single_model = create_cnn_model()
single_model.summary()


/Users/snithshibu/.pyenv/versions/3.10.13/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-12-23 12:50:43.049143: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-12-23 12:50:43.049190: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-12-23 12:50:43.049196: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2025-12-23 12:50:43.049266: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-12-23 12:50:43.049287: I tensorflow/core/common_runtime/pluggable_device/pluggable_d

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 5408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        54,090 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,410 (212.54 KB)

 Trainable params: 54,410 (212.54 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
history_single = single_model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=8,
    validation_data=(x_val, y_val),
    verbose=1
)

val_loss_single, val_acc_single = single_model.evaluate(x_val, y_val, verbose=0)
test_loss_single, test_acc_single = single_model.evaluate(x_test_small, y_test_small, verbose=0)

print("Single model validation accuracy:", val_acc_single)
print("Single model test accuracy:", test_acc_single)


Epoch 1/10


2025-12-23 12:50:56.126443: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 253ms/step - accuracy: 0.0857 - loss: 2.3103 - val_accuracy: 0.2667 - val_loss: 2.0682
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.4857 - loss: 1.9382 - val_accuracy: 0.3333 - val_loss: 1.9042
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6000 - loss: 1.6559 - val_accuracy: 0.2667 - val_loss: 1.7393
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5714 - loss: 1.4132 - val_accuracy: 0.3333 - val_loss: 1.5826
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8000 - loss: 1.1394 - val_accuracy: 0.6000 - val_loss: 1.3759
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8000 - loss: 0.8888 - val_accuracy: 0.6667 - val_loss: 1.2446
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8857 - loss: 0.6940 - val_accuracy: 0.6000 - val_loss: 1.1483
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9143 - loss: 0.5284 - val_accuracy: 0.6000 - val_loss: 1.0694
Epoch 9/10

In [10]:
num_models = 5
epochs_ensemble = 3  
batch_size = 8

ensemble_models = []

n_train = x_train.shape[0]

for i in range(num_models):
    print(f"\nTraining model {i+1}/{num_models}")
    
    indices = np.random.choice(n_train, size=n_train, replace=True)
    x_bootstrap = x_train[indices]
    y_bootstrap = y_train[indices]
    
    model_i = create_cnn_model()
    
    model_i.fit(
        x_bootstrap, y_bootstrap,
        epochs=epochs_ensemble,
        batch_size=batch_size,
        validation_data=(x_val, y_val),
        verbose=1
    )
    
    ensemble_models.append(model_i)



Training model 1/5
Epoch 1/3
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.3714 - loss: 2.2505 - val_accuracy: 0.2667 - val_loss: 2.0878
Epoch 2/3
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6571 - loss: 1.7762 - val_accuracy: 0.2667 - val_loss: 1.9665
Epoch 3/3
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8000 - loss: 1.4282 - val_accuracy: 0.5333 - val_loss: 1.8528

Training model 2/5
Epoch 1/3
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.3143 - loss: 2.1280 - val_accuracy: 0.2000 - val_loss: 2.0655
Epoch 2/3
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3714 - loss: 1.5994 - val_accuracy: 0.3333 - val_loss: 2.0671
Epoch 3/3
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6286 - loss: 1.2864 - val_accuracy: 0.3333 - val_loss: 2.0738

Training model 3/5
Epoch 1/3
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.1714 - loss: 2.1469 - val_accuracy: 0.0667 - val_loss: 2.2614
Epoch 2/3
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5429 - los

In [11]:
def ensemble_predict_proba(models, x):
    preds_sum = None
    for m in models:
        p = m.predict(x, verbose=0)  
        if preds_sum is None:
            preds_sum = p
        else:
            preds_sum += p
    preds_avg = preds_sum / len(models)
    return preds_avg

def accuracy_from_proba(preds_avg, y_true):
    y_pred = np.argmax(preds_avg, axis=1)
    return np.mean(y_pred == y_true)


In [12]:
val_proba_ens = ensemble_predict_proba(ensemble_models, x_val)
val_acc_ensemble = accuracy_from_proba(val_proba_ens, y_val)

test_proba_ens = ensemble_predict_proba(ensemble_models, x_test_small)
test_acc_ensemble = accuracy_from_proba(test_proba_ens, y_test_small)

print("Ensemble model validation accuracy:", val_acc_ensemble)
print("Ensemble model test accuracy:", test_acc_ensemble)

print("Single model validation accuracy:", val_acc_single)
print("Single model test accuracy:", test_acc_single)


Ensemble model validation accuracy: 0.26666666666666666
Ensemble model test accuracy: 0.42
Single model validation accuracy: 0.6666666865348816
Single model test accuracy: 0.6000000238418579


## Results: Single CNN vs Ensemble CNNs

Using the Fashion-MNIST dataset (first 50 samples for train and test), images were normalized to [0, 1] and reshaped to (28, 28, 1). A CNN with Conv2D(32, 3×3, ReLU) → MaxPooling2D(2×2) → Flatten → Dense(10, softmax) was trained with Adam and sparse categorical cross-entropy. 

**Single model performance:**
- Validation accuracy: ~0.67  
- Test accuracy: ~0.60  

**Ensemble (5 CNNs with bootstrap sampling, 3 epochs each):**
- Validation accuracy: ~0.27  
- Test accuracy: ~0.42  

In this small-data setting (only 35 train samples after split), the single model outperformed the ensemble. The ensemble’s bootstrap resampling reduced the effective unique data seen by each model, and averaging their predictions did not compensate for the high variance and underfitting. With a larger training set, ensembles typically outperform single models due to variance reduction. 
